In [1]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier, export_text, plot_tree, sns
import constants
import utils

In [ ]:
data

Plot data, shows how often MonkeyPox is negative and positive

In [ ]:
series = data[constants.MONKEY_POX].value_counts()
#series2,
# TODO: Try to use countplot and change colors to normal blue and orange
plt.bar(series.index, series.values, color=constants.COLOR_PALETTE)
plt.show()

Plots heatmap: Shows correlations between the different features

In [ ]:
sns.heatmap(data.corr(), annot=True)
plt.show()

Shows how often each systemic illness is causing MonkeyPox to be positive or negative

In [2]:
# TODO: Change legend and axises
#test_df = data.groupby(by=[constants.SYSTEMIC_ILLNESS,constants.MONKEY_POX])[constants.SYSTEMIC_ILLNESS].count()
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX], palette=constants.COLOR_PALETTE)
plt.show()

Plots data, using pairplot

In [ ]:
#sns.pairplot(data.head(100), hue='MonkeyPox')

Calculates accuracy, using GaussianNB()

In [ ]:
simple_dict = {'None': 1, 'Fever': 2, 'Swollen Lymph Nodes': 3, 'Muscle Aches and Pain': 4}
data[constants.SYSTEMIC_ILLNESS] = [simple_dict[item] for item in data[constants.SYSTEMIC_ILLNESS]]
data_tree = data.iloc[:,:-1]
target = data.iloc[:,-1:]
ran_stream = 23
x_train, x_test, y_train, y_test = train_test_split(data_tree, target, random_state=ran_stream)

model = GaussianNB()
model.fit(x_train, y_train)
y_prediction = model.predict(x_test)
print(100 * accuracy_score(y_test, y_prediction))

Creates DecisionTree, by using entropy

In [ ]:
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=5)
model_tree.fit(data_tree, target)
text_representation = export_text(model_tree, feature_names=data_tree.columns.values.tolist())
figure = plt.figure(figsize=(10,8))
plot_tree(model_tree, feature_names=data_tree.columns, filled=True, rounded=True)
plt.show()


Plots data, when symptom (2-7) is True and MonkeyPox is Positive

In [ ]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.fetch_true_symptoms(symptoms, symptoms_df)
print(symptoms_df)

pos_lst, negative_lst = utils.create_outcome_lists(symptoms, symptoms_df)
df1 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': pos_lst})
df2 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': negative_lst})
df1['outcome'] = 'Positive'
df2['outcome'] = 'Negative'
res = pd.concat([df1, df2])
sns.barplot(x='Symptoms', y='Number of infections', data=res, hue='outcome', palette=constants.COLOR_PALETTE)
plt.show()